In [1]:
import numpy as np
import pandas as pd
import torch
import gc
import time

import syft as sy
from syft.core.adp.entity import Entity

In [2]:
import os
os.getcwd()

'/home/azureuser/PySyft/notebooks/twitter'

In [3]:
df = pd.read_parquet("../../data/synthetic_data/1.4M rows dataset/1.4M_rows_dataset_sample.parquet")
df.head()

,tweet_id,impressions,tweet_date_time,date,time,user_id,url,publication_title,ad_fontes_bias,ad_fontes_reliability,domain
0,1,7245,2020-04-02 14:44:33,2020-04-02,14:44:33,360567,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
1,1,7245,2020-04-02 14:44:33,2020-04-02,14:44:33,675832,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
2,1,7245,2020-04-02 14:44:33,2020-04-02,14:44:33,501687,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
3,1,7245,2020-04-02 14:44:33,2020-04-02,14:44:33,191111,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
4,1,7245,2020-04-02 14:44:33,2020-04-02,14:44:33,283780,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com


In [4]:
df.shape

(1486698, 11)

In [5]:
# Let's login into the domain node
domain_node = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into canada... done!


In [17]:
def upload_subset(start_index, end_index, count):
    
    name = "Tweets- 1.4M rows " + str(count)
    impressions = ((np.array(list(df['impressions'][start_index: end_index])))).astype(np.int32)
    publication_title = ((list(df['publication_title'][start_index: end_index])))

    entities = list()
    for i in range(len(publication_title)):
        entities.append(Entity(name=publication_title[i]))
        
    tweets_data = sy.Tensor(impressions).private(min_val=0, max_val=30, entities = entities)
    
    domain_node.load_dataset(
    assets={"tweets": tweets_data},
    name=name,
    description="""Tweets- 1.4 M rows 500k Jan 26th 5 pm""",
    )

In [10]:
dfd1 = df.head(250000)
dfd2 = df.head(500000)
dfd3 = df.head(750000)
dfd4 = df.head(1000000)

In [18]:
start_time = time.time()

count = 0
last_val = 0
for i in range(0,dfd2.shape[0]-250000,250000):
    count = count + 1
    last_val += 250000
    upload_subset(i, i + 250000, count)
    
# last subset
upload_subset(last_val, dfd2.shape[0], count + 1)

print("--- %s seconds ---" % (time.time() - start_time))

Loading dataset... uploading... SUCCESS!                                                                                                                                      

Run <your client variable>.datasets to see your new dataset loaded into your machine!
Loading dataset... uploading... SUCCESS!                                                                                                                                      

Run <your client variable>.datasets to see your new dataset loaded into your machine!
--- 296.72641229629517 seconds ---


In [19]:
domain_node.datasets

Idx,Name,Description,Assets,Id
[0],Tweets- 1M rows part 1,Tweets- 1M rows,,b809e091-ddcb-49af-88d3-abbc8dd79ec8
[1],Tweets- 1M rows part 2,Tweets- 1M rows,,73608fb2-eba1-4d44-ae7b-e7b3582c3d42
[2],Tweets- 1M rows part 3,Tweets- 1M rows,,e6cdab22-9fee-4140-a17e-c0a4d64fb52b
[3],Tweets- 1M rows part 4,Tweets- 1M rows,,3d28116e-f847-463a-a488-c8b92cfd0528
[4],Tweets- 1M rows part 5,Tweets- 1M rows,,0302ca3b-9676-4de5-9fb6-f890ff6a8991
[5],Tweets- 1M rows,"150 tweets, 10K impressions",,ba0e8fdd-d0bd-4f1a-949a-c43fe91520cf
[6],Tweets- 100K rows,"150 tweets, 1K impressions",,e3d3f4f8-b342-4baa-bffd-633cd0a01df2
[7],Tweets- 1K rows 1,"Tweets- 159,671 rows","[""tweets""] -> Tensor",7784079c-db0c-4a13-bd45-91e178206a62
[8],Tweets- 10K impressions 1,"Tweets- 1,506,029 rows",,bf8d0bef-d360-4e32-b9a0-44f5ac26c8dc
[9],Tweets- 10K impressions 1,"Tweets- 1,506,029 rows",,90bcb170-129a-407a-830c-97c12568ad49


In [ ]:
start_time = time.time()

res=0 
for i in range(51,53):
    dataset = domain_node.datasets[i]["tweets"]
    res += dataset.sum(axis=1)

print("--- %s seconds ---" % (time.time() - start_time))